In [70]:
import numpy as np
import tensorflow as tf

Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 100000  # Number of simualted paths
m = 50      # number of exercise dates
T = 1       # maturity
order = 6   # Polynmial order
Δt = T / m  # interval between two exercise dates


# In TensorFlow you can use tf.one_hot, so you dont need to convert that
def one_hot(array, depth):
    return tf.one_hot(array, depth)
    

# Construct polynomial features of order up to k using the
# recursive formulation
def chebyshev_basis(x, k):
    #B = [np.ones(len(x)), x]
    #for n in range(2, k):
    #    Bn = 2 * x * B[n - 1] - B[n - 2]
    #    B.append(Bn)

    #return np.column_stack(B)

    B = tf.ones((1, x.shape[0]))

    x = tf.expand_dims(x,1)
    x = tf.transpose(x)
    B = tf.concat([B, x], axis=0)
    
    for n in range(2, k):        
        temp = 2 * x * B[n - 1, :] - B[n - 2, :]
        B = tf.concat([B, temp], axis=0)
        result=tf.transpose(B)

    return result
    
    

# scales x to be in the interval(-1, 1)
def scale(x):


    xmin = tf.math.reduce_min(x)
    xmax = tf.math.reduce_max(x)
    a = 2 / (xmax - xmin)
    b = 1 - a * xmax
    return a * x + b
    

# simulates the stock price evolution
def advance(S, r, σ, Δt, n):
    #dB = np.sqrt(Δt) * np.random.normal(size=S.size)
    #out = S + r * S * Δt + σ * S * dB
    
    dB = tf.sqrt(Δt) * tf.random.normal(shape=[n])
    out = S + tf.constant(r, dtype='float') * S * tf.constant(Δt) + tf.constant(σ) * S * dB

    
    return out


# LSMC algorithm
def compute_price(order, Spot, σ, K, r):
    tf.random.set_seed(0)
    
    #S0 = Spot * np.ones(n)
    S0 = Spot * tf.ones(n)
    S = {0: S0}

    for t in range(m):
        S[t + 1] = advance(S[t], r, σ, Δt, n)
        

    discount = tf.exp(-r * Δt)
    CFL = {t: tf.maximum(0., K - S[t]) for t in S.keys()}
    value_tp1 = CFL[m] * discount
    CV = {m: tf.zeros_like(S0)}

    
    
    for t in range(m - 1, 0, -1):
        
        X = chebyshev_basis(scale(S[t]), order)
        Y = value_tp1

        # regression to estimate the continuation value
        #Θ = np.linalg.solve(X.T @ X, X.T @ Y)
        
        XT = tf.transpose(X)
        XTX = tf.linalg.matmul(XT, X)
        XTY = tf.linalg.matvec(XT, Y)
        XTY = tf.expand_dims(XTY, 1)
        
        Θ = tf.linalg.solve(XTX, XTY)
        
        CV[t] = X @ Θ
        
        
        CV[t] = tf.squeeze(CV[t])
        exercise = CFL[t] > CV[t]
            
        value_t = tf.where(exercise, CFL[t], value_tp1)
        value_tp1 = discount * value_t

    POF = {t: tf.where(CV[t] < CFL[t], CFL[t], 0) for t in range(1, m + 1)}
    
    pof_values = tf.convert_to_tensor(list(POF.values()))
    POF = tf.stack(pof_values)
    
    idx_payoffs = tf.argmax(POF > 0, axis=0)
    FPOF = tf.transpose(one_hot(idx_payoffs, m)) * POF
    
    m_range = tf.range(m)
    m_range = tf.expand_dims(m_range, axis=1)
    m_range = tf.dtypes.cast(m_range, 'float')
    
    dFPOF = FPOF * tf.exp(-r * m_range * Δt)
    PRICE = tf.math.reduce_sum(dFPOF) / n
    return PRICE



#print(compute_price(order, Spot, σ, K, r))
test = compute_price(order, Spot, σ, K, r)

print(test)

49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
tf.Tensor(4.4873776, shape=(), dtype=float32)
